In [1]:
import cirq
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
import time
import sympy

torch.set_default_dtype(torch.float64)

In [4]:
# Configuration
n_qubits = 2
noise_type = "depolarizing"  # Change to "bitflip", "phaseflip", "amplitude_damping"
noise_strength = 0.05

# Data preparation
iris = datasets.load_iris()
X = iris.data[:, :n_qubits]  # only first n_qubits features
y = iris.target

# Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train, dtype=torch.float64)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test, dtype=torch.float64)

# Create qubits
qubits = [cirq.GridQubit(0, i) for i in range(n_qubits)]

class CirqQuantumLayer(torch.nn.Module):
    """
    A PyTorch layer that wraps a Cirq quantum circuit
    """
    def __init__(self, n_qubits, n_layers=1):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.qubits = [cirq.GridQubit(0, i) for i in range(n_qubits)]
        
        # Initialize quantum parameters as PyTorch parameters
        # Shape: (n_layers, n_qubits, 3) for strongly entangling layers
        self.quantum_weights = torch.nn.Parameter(
            torch.randn(n_layers, n_qubits, 3, dtype=torch.float64) * 0.5
        )
        
        # Use density matrix simulator for noise
        self.simulator = cirq.DensityMatrixSimulator()
        
    def add_noise_layer(self, circuit):
        """Add noise to all qubits according to noise_type"""
        if noise_type == "depolarizing":
            for qubit in self.qubits:
                circuit.append(cirq.depolarize(noise_strength)(qubit))
        elif noise_type == "bitflip":
            for qubit in self.qubits:
                circuit.append(cirq.bit_flip(noise_strength)(qubit))
        elif noise_type == "phaseflip":
            for qubit in self.qubits:
                circuit.append(cirq.phase_flip(noise_strength)(qubit))
        elif noise_type == "amplitude_damping":
            for qubit in self.qubits:
                circuit.append(cirq.amplitude_damp(noise_strength)(qubit))
    
    def angle_embedding(self, circuit, inputs):
        """Embed classical data as rotation angles"""
        # Convert tensor to numpy if needed
        if isinstance(inputs, torch.Tensor):
            inputs = inputs.detach().cpu().numpy()
        for i, qubit in enumerate(self.qubits):
            circuit.append(cirq.ry(float(inputs[i]))(qubit))
    
    def strongly_entangling_layer(self, circuit, weights, layer_idx):
        """Implement strongly entangling layer"""
        # Convert tensor to numpy if needed
        if isinstance(weights, torch.Tensor):
            weights = weights.detach().cpu().numpy()
            
        # Apply rotations
        for i, qubit in enumerate(self.qubits):
            circuit.append(cirq.rx(float(weights[layer_idx, i, 0]))(qubit))
            circuit.append(cirq.ry(float(weights[layer_idx, i, 1]))(qubit))
            circuit.append(cirq.rz(float(weights[layer_idx, i, 2]))(qubit))
        
        # Add entangling gates
        for i in range(self.n_qubits):
            control = self.qubits[i]
            target = self.qubits[(i + 1) % self.n_qubits]
            circuit.append(cirq.CNOT(control, target))
    
    def create_circuit(self, inputs, weights):
        """Create the complete quantum circuit"""
        circuit = cirq.Circuit()
        
        # Angle embedding
        self.angle_embedding(circuit, inputs)
        
        # Add noise after embedding
        self.add_noise_layer(circuit)
        
        # Strongly entangling layers
        for layer in range(self.n_layers):
            self.strongly_entangling_layer(circuit, weights, layer)
        
        # Add noise after ansatz
        self.add_noise_layer(circuit)
        
        return circuit
    
    def get_expectation_values(self, circuit):
        """Calculate expectation values of Pauli-Z on all qubits"""
        result = self.simulator.simulate(circuit)
        expectations = []
        
        for i, qubit in enumerate(self.qubits):
            z_op = cirq.Z(qubit)
            expectation = z_op.expectation_from_density_matrix(
                result.final_density_matrix,
                qubit_map={q: j for j, q in enumerate(self.qubits)}
            ).real
            expectations.append(expectation)
        
        return torch.tensor(expectations, dtype=torch.float64)
    
    def forward(self, x):
        """Forward pass through the quantum layer"""
        batch_size = x.shape[0]
        outputs = []
        
        for i in range(batch_size):
            # Convert input to numpy for Cirq
            x_numpy = x[i].detach().cpu().numpy()
            weights_numpy = self.quantum_weights.detach().cpu().numpy()
            
            # Create circuit for this input
            circuit = self.create_circuit(x_numpy, weights_numpy)
            
            # Get expectation values
            expectations = self.get_expectation_values(circuit)
            outputs.append(expectations)
        
        return torch.stack(outputs)

class CirqQuantumLayerDifferentiable(CirqQuantumLayer):
    """
    Differentiable version using finite differences for gradients
    """
    def __init__(self, n_qubits, n_layers=1, epsilon=0.01):
        super().__init__(n_qubits, n_layers)
        self.epsilon = epsilon
    
    def forward(self, x):
        # Define a function that computes the quantum circuit output
        def quantum_function(weights_flat, x_input):
            weights_reshaped = weights_flat.view(self.n_layers, self.n_qubits, 3)
            # Convert tensors to numpy for Cirq
            x_numpy = x_input.detach().cpu().numpy() if isinstance(x_input, torch.Tensor) else x_input
            weights_numpy = weights_reshaped.detach().cpu().numpy()
            circuit = self.create_circuit(x_numpy, weights_numpy)
            expectations = self.get_expectation_values(circuit)
            return expectations
        
        # Enable gradients using autograd.Function
        return QuantumFunction.apply(quantum_function, self.quantum_weights.flatten(), x, self.quantum_weights.shape)

class QuantumFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, quantum_func, weights_flat, x, weight_shape):
        ctx.quantum_func = quantum_func
        ctx.weight_shape = weight_shape
        ctx.save_for_backward(weights_flat, x)
        
        batch_size = x.shape[0]
        outputs = []
        
        for i in range(batch_size):
            # Convert to numpy for Cirq
            x_numpy = x[i].detach().cpu().numpy() if isinstance(x[i], torch.Tensor) else x[i]
            output = quantum_func(weights_flat, x_numpy)
            outputs.append(output)
        
        return torch.stack(outputs)
    
    @staticmethod
    def backward(ctx, grad_output):
        quantum_func = ctx.quantum_func
        weights_flat, x = ctx.saved_tensors
        weight_shape = ctx.weight_shape
        
        # Finite difference for gradients
        epsilon = 0.01
        grad_weights = torch.zeros_like(weights_flat)
        
        # Only compute gradients for weights (not for x)
        batch_size = x.shape[0]
        
        for param_idx in range(len(weights_flat)):
            # Forward difference
            weights_plus = weights_flat.clone()
            weights_plus[param_idx] += epsilon
            
            weights_minus = weights_flat.clone()
            weights_minus[param_idx] -= epsilon
            
            outputs_plus = []
            outputs_minus = []
            
            for batch_idx in range(batch_size):
                # Convert to numpy for Cirq
                x_numpy = x[batch_idx].detach().cpu().numpy()
                out_plus = quantum_func(weights_plus, x_numpy)
                out_minus = quantum_func(weights_minus, x_numpy)
                outputs_plus.append(out_plus)
                outputs_minus.append(out_minus)
            
            outputs_plus = torch.stack(outputs_plus)
            outputs_minus = torch.stack(outputs_minus)
            
            # Finite difference gradient
            finite_diff = (outputs_plus - outputs_minus) / (2 * epsilon)
            
            # Apply chain rule
            grad_weights[param_idx] = torch.sum(grad_output * finite_diff)
        
        return None, grad_weights, None, None

class HQNN(nn.Module):
    """Hybrid Quantum Neural Network with Cirq backend"""
    def __init__(self):
        super().__init__()
        self.quantum_layer = CirqQuantumLayerDifferentiable(n_qubits, n_layers=1)
        self.classical_layer = nn.Linear(n_qubits, 1)
        
    def forward(self, x):
        quantum_output = self.quantum_layer(x)
        return self.classical_layer(quantum_output)


In [5]:
# Create model
model = HQNN()
model = model.to(dtype=torch.float64)

# Training
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

epochs = 30
print("Starting training...")

for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Forward pass
    preds = model(X_train)
    loss = loss_fn(preds.squeeze(), y_train)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}")

print("Training completed!")


Starting training...
Epoch 5/30, Loss: 0.585217
Epoch 10/30, Loss: 0.559069
Epoch 15/30, Loss: 0.542188
Epoch 20/30, Loss: 0.529044
Epoch 25/30, Loss: 0.517563
Epoch 30/30, Loss: 0.507608
Training completed!


In [6]:
# Evaluation
print("\nEvaluating on test set...")
with torch.no_grad():
    start_time = time.time()
    preds = model(X_test).detach().cpu().numpy()
    end_time = time.time()

# Convert predictions to binary (threshold at 0.5 for multi-class, we'll use argmax instead)
preds_rounded = np.round(preds).astype(int).flatten()
preds_rounded = np.clip(preds_rounded, 0, 2)  # Clip to valid class range

# Ensure y_test is integer
y_true = y_test.detach().cpu().numpy().astype(int)

# Metrics
mse = mean_squared_error(y_true, preds.flatten())
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_true, preds.flatten())

# For F1-score, we need proper classification predictions
try:
    f1 = f1_score(y_true, preds_rounded, average='macro')
except:
    f1 = 0.0  # If F1 calculation fails

print("\n--- Evaluation Metrics ---")
print(f"Noise type       : {noise_type}")
print(f"Noise strength   : {noise_strength}")
print(f"Number of qubits : {n_qubits}")
print(f"MSE              : {mse:.6f}")
print(f"RMSE             : {rmse:.6f}")
print(f"MAE              : {mae:.6f}")
print(f"F1-score         : {f1:.6f}")
print(f"epochs           : {epochs}")
print(f"Inference time   : {(end_time - start_time):.6f} seconds")


Evaluating on test set...

--- Evaluation Metrics ---
Noise type       : depolarizing
Noise strength   : 0.05
Number of qubits : 2
MSE              : 0.618277
RMSE             : 0.786306
MAE              : 0.686132
F1-score         : 0.269231
epochs           : 30
Inference time   : 0.083533 seconds
